In [11]:
from naima.models import (
    ExponentialCutoffPowerLaw,
    ExponentialCutoffBrokenPowerLaw,    
    Synchrotron,
    InverseCompton,
    Bremsstrahlung,
)
import numpy as np
import naima
import matplotlib.pyplot as plt
from astropy.constants import c
import astropy.units as u
import ipywidgets as widgets
from ipywidgets import interact

In [2]:
# Define energy range
energy_range = np.logspace(8, 13, 100) * u.eV

# Function to update plot based on distance and e_cutoff
def update_plot_pion_decay(distance_kpc, e_cutoff_GeV, n_target_cm3):
    """
    Function to feed interactive widgets to update plot of the 
    SED for a pion decay model. You can change the distance,
    cutoff energy and target density.

    Parameters
    ----------
    distance_kpc : float
        Distance to the source in kpc.
    e_cutoff_GeV : float
        Cutoff energy in GeV.
    n_target_cm3 : float
        Target density in cm^-3.
    """
    # Convert input values to astropy units
    distance = distance_kpc * u.kpc
    e_cutoff = e_cutoff_GeV * u.GeV
    n_target = n_target_cm3 * u.cm ** -3
    
    # Define particle distribution with the given cutoff energy
    part_dist = naima.models.ExponentialCutoffPowerLaw(
        amplitude=8e31 / u.eV,
        e_0=130 * u.GeV,
        alpha=2.44,
        e_cutoff=e_cutoff
    )
    
    # Create radiation pion decay model
    rad_models = naima.models.PionDecay(part_dist, nh=n_target)
    
    # Compute SED
    sed = rad_models.sed(energy_range, distance=distance)

    # Clear previous figure and plot new SED
    plt.figure(figsize=(8, 6))
    plt.loglog(
        energy_range, sed,
        label=f"Distance = {distance_kpc:.2f} kpc,\n"+\
                f"E$_{{\\rm cutoff}}$ = {e_cutoff_GeV:.0f} GeV,\n"+\
                f"n$_{{\\rm h}}$ = {n_target_cm3:.1e} cm$^{{-3}}$"
    )
    plt.xlabel("Energy (eV)")
    plt.ylabel("E$^2$ d$\phi$/dE (erg cm$^{-2}$ s$^{-1}$)")
    plt.legend()
    plt.grid(True, which="both", linestyle="--", alpha=0.6)
    plt.ylim(1e-13, 1e-9)
    plt.show()

distance_slider=widgets.FloatLogSlider(
    value=1.4, min=-1, max=1, step=0.1, base=10, 
    description="Distance (kpc)",
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

e_cutoff_slider=widgets.FloatLogSlider(
    value=280, min=1, max=4, step=0.1, base=10, 
    description="E_cutoff (GeV)",
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

target_density_slider=widgets.FloatLogSlider(
    value=1.4, min=4, max=12, step=1, base=10, 
    description=r"n_target (cm-3)",
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

# Create interactive widgets
interact(update_plot_pion_decay, 
         distance_kpc=distance_slider,
         e_cutoff_GeV=e_cutoff_slider,
         n_target_cm3=target_density_slider
);


interactive(children=(FloatLogSlider(value=1.4, description='Distance (kpc)', layout=Layout(width='400px'), ma…

In [3]:
# Define energy range
energy_range = np.logspace(8, 13, 100) * u.eV

# Function to update plot based on distance, e_cutoff, and photon field
def update_plot_inverse_compton(distance_kpc, e_cutoff_GeV, photon_field):
    """
    Function to feed interactive widgets to update plot of the 
    SED for an Inverse Compton model. You can change the distance,
    cutoff energy, and seed photon field.

    Parameters
    ----------
    distance_kpc : float
        Distance to the source in kpc.
    e_cutoff_GeV : float
        Cutoff energy in GeV.
    photon_field : str
        Seed photon field for Inverse Compton interaction.
    """
    # Convert input values to astropy units
    distance = distance_kpc * u.kpc
    e_cutoff = e_cutoff_GeV * u.GeV
    photon_seed = photon_field  # No conversion needed
    
    # Define particle distribution with the given cutoff energy
    part_dist = naima.models.ExponentialCutoffPowerLaw(
        amplitude=1e36 / u.eV,
        e_0=1 * u.TeV,
        alpha=2.1,
        e_cutoff=e_cutoff
    )
    
    # Create radiation model for Inverse Compton
    rad_models = naima.models.InverseCompton(part_dist, seed_photon_fields=[photon_seed])
    
    # Compute SED
    sed = rad_models.sed(energy_range, distance=distance)

    # Clear previous figure and plot new SED
    plt.figure(figsize=(8, 6))
    plt.loglog(
        energy_range, sed,
        label=f"Distance = {distance_kpc:.2f} kpc,\n"+\
              f"E$_{{\\rm cutoff}}$ = {e_cutoff_GeV:.0f} GeV,\n"+\
              f"Photon Seed = {photon_seed}"
    )
    plt.xlabel("Energy (eV)")
    plt.ylabel("E$^2$ d$\phi$/dE (erg cm$^{-2}$ s$^{-1}$)")
    plt.legend()
    plt.grid(True, which="both", linestyle="--", alpha=0.6)
    plt.ylim(1e-13, 1e-9)
    plt.show()

# Create interactive widgets
distance_slider = widgets.FloatLogSlider(
    value=1.4, min=-1, max=1, step=0.1, base=10, 
    description="Distance (kpc)",
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

e_cutoff_slider = widgets.FloatLogSlider(
    value=280, min=1, max=4, step=0.1, base=10, 
    description="E_cutoff (GeV)",
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

photon_field_dropdown = widgets.Dropdown(
    options=['CMB', 'FIR', 'NIR'],
    value='CMB',
    description="Photon Seed:",
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)

# Create interactive widgets
interact(update_plot_inverse_compton, 
         distance_kpc=distance_slider,
         e_cutoff_GeV=e_cutoff_slider,
         photon_field=photon_field_dropdown
);


interactive(children=(FloatLogSlider(value=1.4, description='Distance (kpc)', layout=Layout(width='400px'), ma…

In [49]:
# Define energy range
energy_range = np.logspace(8, 13, 100) * u.eV

# Function to update plot based on distance, e_cutoff, and target density
def update_plot_bremsstrahlung(distance_kpc, e_cutoff_GeV, n_target_cm3):
    """
    Function to feed interactive widgets to update the plot of the 
    SED for a Bremsstrahlung model. You can change the distance,
    cutoff energy, and target density.

    Parameters
    ----------
    distance_kpc : float
        Distance to the source in kpc.
    e_cutoff_GeV : float
        Cutoff energy in GeV.
    n_target_cm3 : float
        Target density in cm^-3.
    """
    # Convert input values to astropy units
    distance = distance_kpc * u.kpc
    e_cutoff = e_cutoff_GeV * u.GeV
    n_target = n_target_cm3 * u.cm ** -3
    
    # Define particle distribution with the given cutoff energy
    part_dist = naima.models.ExponentialCutoffPowerLaw(
        amplitude=8e31 / u.eV,
        e_0=130 * u.GeV,
        alpha=2.44,
        e_cutoff=e_cutoff
    )
    
    # Create radiation model for Bremsstrahlung
    rad_models = naima.models.Bremsstrahlung(part_dist, n0=n_target)
    
    # Compute SED
    sed = rad_models.sed(energy_range, distance=distance)

    # Clear previous figure and plot new SED
    plt.figure(figsize=(8, 6))
    plt.loglog(
        energy_range, sed,
        label=f"Distance = {distance_kpc:.2f} kpc,\n"+\
              f"E$_{{\\rm cutoff}}$ = {e_cutoff_GeV:.0f} GeV,\n"+\
              f"n$_{{\\rm 0}}$ = {n_target_cm3:.1e} cm$^{{-3}}$"
    )
    plt.xlabel("Energy (eV)")
    plt.ylabel("E$^2$ d$\phi$/dE (erg cm$^{-2}$ s$^{-1}$)")
    plt.legend()
    plt.grid(True, which="both", linestyle="--", alpha=0.6)
    plt.ylim(1e-13, 1e-9)
    plt.show()

# Create interactive widgets
distance_slider = widgets.FloatLogSlider(
    value=1.4, min=-1, max=1, step=0.1, base=10, 
    description="Distance (kpc)",
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

e_cutoff_slider = widgets.FloatLogSlider(
    value=280, min=1, max=4, step=0.1, base=10, 
    description="E_cutoff (GeV)",
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

n_target_slider = widgets.FloatLogSlider(
    value=1.4, min=5, max=10, step=0.1, base=10, 
    description="n_0 (cm-3)",
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

# Create interactive widgets
interact(update_plot_bremsstrahlung, 
         distance_kpc=distance_slider,
         e_cutoff_GeV=e_cutoff_slider,
         n_target_cm3=n_target_slider
);


interactive(children=(FloatLogSlider(value=1.4, description='Distance (kpc)', layout=Layout(width='400px'), ma…

In [50]:
# Function to update plot based on distance, e_cutoff, and magnetic field
def update_plot_synch_ic(distance_kpc, e_cutoff_GeV, B_uGauss):
    """
    Function to update the plot of the SED for a combined 
    Synchrotron and Inverse Compton (IC) model. You can change 
    the distance, cutoff energy, and magnetic field strength.

    Parameters
    ----------
    distance_kpc : float
        Distance to the source in kpc.
    e_cutoff_GeV : float
        Cutoff energy in GeV.
    B_uGauss : float
        Magnetic field strength in microGauss.
    """
    # Convert input values to astropy units
    distance = distance_kpc * u.kpc
    e_cutoff = e_cutoff_GeV * u.GeV
    B = B_uGauss * u.uG
    
    # Define particle distribution with updated e_cutoff
    part_dist = naima.models.ExponentialCutoffPowerLaw(
        amplitude=1e36 / u.eV,
        e_0=1 * u.TeV,
        alpha=2.1,
        e_cutoff=e_cutoff
    )
    
    # Compute Synchrotron emission
    SYN = naima.models.Synchrotron(part_dist, B=B)
    
    # Define energy array for synchrotron seed photon field
    Esy = np.logspace(-6, 6, 100) * u.eV
    Lsy = SYN.flux(Esy, distance=0 * u.cm)

    # Define source radius and compute photon density
    R = 2 * u.pc
    # The factor 2.24 accounts for geometrical considerations of a uniform spherical emitter
    phn_sy = Lsy / (4 * np.pi * R**2 * c) * 2.24

    # Compute Inverse Compton emission with multiple seed photon fields
    IC = naima.models.InverseCompton(part_dist, seed_photon_fields=['CMB', 'FIR', 'NIR', ['SSC', Esy, phn_sy]])

    # Compute SEDs
    spectrum_energy = np.logspace(-10, 14, 100) * u.eV
    sed_IC = IC.sed(spectrum_energy, distance=distance)
    sed_SYN = SYN.sed(spectrum_energy, distance=distance)

    # Clear previous figure and plot new SED
    plt.figure(figsize=(8, 6))
    plt.loglog(spectrum_energy, sed_SYN, label="Synchrotron")
    plt.loglog(spectrum_energy, sed_IC, label="Inverse Compton")
    
    plt.xlabel("Energy (eV)")
    plt.ylabel("E$^2$ d$\phi$/dE (erg cm$^{-2}$ s$^{-1}$)")
    plt.legend(title=f"Distance = {distance_kpc:.2f} kpc,\n"
                     f"E$_{{\\rm cutoff}}$ = {e_cutoff_GeV:.0f} GeV,\n"
                     f"B = {B_uGauss:.1f} µG")
    plt.grid(True, which="both", linestyle="--", alpha=0.6)
    plt.ylim(1e-13, 1e-6)

    # plt.plot(np.geomspace(1e-6, 1e6, 100), 1e-11*(np.geomspace(1e-6, 1e6, 100)/1e-6)**(2+(2+1-2.1-2*2)/2))

    plt.show()


# Create interactive widgets
distance_slider = widgets.FloatLogSlider(
    value=1.4, min=-1, max=1, step=0.1, base=10, 
    description="Distance (kpc)",
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

e_cutoff_slider = widgets.FloatLogSlider(
    value=280, min=1, max=4, step=0.1, base=10, 
    description="E_cutoff (GeV)",
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

B_slider = widgets.FloatLogSlider(
    value=100, min=-1, max=4, step=0.1, base=10, 
    description="B (µG)",
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

# Create interactive plot
interact(update_plot_synch_ic, 
         distance_kpc=distance_slider,
         e_cutoff_GeV=e_cutoff_slider,
         B_uGauss=B_slider
);

interactive(children=(FloatLogSlider(value=1.4, description='Distance (kpc)', layout=Layout(width='400px'), ma…

# Let's try to fit by hand our data

In [55]:
from astropy.io import ascii
from astropy.constants import h, c

# Load data
data = ascii.read("../data/CrabNebula/CrabNebula_spectrum.ecsv")

# Function to update plot based on distance, e_cutoff, and magnetic field
def update_plot_synch_ic(distance_kpc, e_cutoff_TeV, B_uGauss):
    """
    Function to update the plot of the SED for a combined 
    Synchrotron and Inverse Compton (IC) model. You can change 
    the distance, cutoff energy, and magnetic field strength.

    Parameters
    ----------
    distance_kpc : float
        Distance to the source in kpc.
    e_cutoff_TeV : float
        Cutoff energy in GeV.
    B_uGauss : float
        Magnetic field strength in microGauss.
    """
    # Convert input values to astropy units
    distance = distance_kpc * u.kpc
    e_cutoff = e_cutoff_TeV * u.TeV
    B = B_uGauss * u.uG
    
    # Define particle distribution with updated e_cutoff
    part_dist = ExponentialCutoffBrokenPowerLaw(
        amplitude=3.699e36 / u.eV,
        e_0=1 * u.TeV,
        e_break=0.265 * u.TeV,
        alpha_1=1.5,
        alpha_2=3.233,
        e_cutoff=e_cutoff,
        beta=2.0,
    )
    
    # Compute Synchrotron emission
    SYN = naima.models.Synchrotron(part_dist, B=B)
    
    # Define energy array for synchrotron seed photon field
    Esy = np.logspace(-6, 6, 100) * u.eV
    Lsy = SYN.flux(Esy, distance=0 * u.cm)

    # Define source radius and compute photon density
    R = 2 * u.pc
    phn_sy = Lsy / (4 * np.pi * R**2 * c) * 2.24

    # Compute Inverse Compton emission with multiple seed photon fields
    IC = naima.models.InverseCompton(part_dist, seed_photon_fields=['CMB', 'FIR', 'NIR', ['SSC', Esy, phn_sy]])

    # Compute SEDs
    spectrum_energy = np.logspace(-10, 14, 100) * u.eV
    sed_IC = IC.sed(spectrum_energy, distance=distance)
    sed_SYN = SYN.sed(spectrum_energy, distance=distance)

    # Convert energy to frequency using ν = E / h
    spectrum_freq = (spectrum_energy / h).to(u.Hz)

    # Convert frequency to wavelength using λ = c / ν
    spectrum_wavelength = (c / spectrum_freq).to(u.m)

    # Create figure
    fig, ax1 = plt.subplots(figsize=(8, 6))
    naima.plot_data(data, e_unit=u.eV, figure=fig)

    # Plot SED
    ax1.loglog(spectrum_energy, sed_SYN, label="Synchrotron")
    ax1.loglog(spectrum_energy, sed_IC, label="Inverse Compton")

    # Format energy axis
    ax1.set_xlabel("Energy (eV)")
    ax1.set_ylabel("E$^2$ d$\phi$/dE (erg cm$^{-2}$ s$^{-1}$)")
    ax1.legend(title=f"Distance = {distance_kpc:.2f} kpc,\n"
                     f"E$_{{\\rm cutoff}}$ = {e_cutoff_TeV:.0f} TeV,\n"
                     f"B = {B_uGauss:.1f} µG")
    ax1.grid(True, which="both", linestyle="--", alpha=0.6)
    ax1.set_ylim(1e-13, 1e-6)

    # Create a secondary x-axis for frequency
    ax2 = ax1.twiny()
    ax2.set_xscale("log")
    ax2.set_xlim(spectrum_freq[0].value, spectrum_freq[-1].value)
    ax2.set_xlabel("Frequency (Hz)")

    # Create a third x-axis for wavelength
    ax3 = ax1.twiny()
    ax3.set_xscale("log")
    ax3.set_xlim(spectrum_wavelength[0].value, spectrum_wavelength[-1].value)
    ax3.spines['top'].set_position(('outward', 40)) 
    ax3.set_xlabel("Wavelength (m)")

    plt.show()

# Create interactive widgets
distance_slider = widgets.FloatLogSlider(
    value=1.4, min=-1, max=1, step=0.1, base=10, 
    description="Distance (kpc)",
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

e_cutoff_slider = widgets.FloatLogSlider(
    value=280, min=1, max=5, step=0.1, base=10, 
    description="E_cutoff (TeV)",
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

B_slider = widgets.FloatLogSlider(
    value=100, min=-1, max=4, step=0.1, base=10, 
    description="B (µG)",
    style={'description_width': '100px'},
    layout=widgets.Layout(width='400px')
)

# Create interactive plot
interact(update_plot_synch_ic, 
         distance_kpc=distance_slider,
         e_cutoff_TeV=e_cutoff_slider,
         B_uGauss=B_slider
);


interactive(children=(FloatLogSlider(value=1.4, description='Distance (kpc)', layout=Layout(width='400px'), ma…